In [422]:
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
import re as re
import os
import copy
import statsmodels.formula.api as sm

In [423]:
conflict = pd.read_csv('Data\\conflict_data\\DRC_2.csv', header = 0, index_col = 0)

aid = pd.read_csv('Data\\aid_data\\data\\level_1a.csv')
aid = aid.dropna(subset=['latitude', 'longitude', 'transactions_start_year']) # drop those entries that don't have coordinates and start year to them
#aid = aid.drop(aid[aid['transactions_start_year'] == ' '].index) # some projects don't have start year, so we drop those rows
aid = aid.drop(aid[aid['even_split_commitments'].isnull()].index) # some projects don't have info on commitments, so we drop those rows
aid['transactions_start_year'] = pd.to_numeric(aid['transactions_start_year'])

worker_deaths = pd.read_csv('Data\\security_incidents.csv', encoding = "ISO-8859-1")

provinces = gpd.read_file('Data\\gadm36_COD_shp/gadm36_COD_1.shp')

In [424]:
geometry = [Point(xy) for xy in zip(conflict.longitude, conflict.latitude)]
gconflict = gpd.GeoDataFrame(conflict, crs = {'init': 'epsg:4326'}, geometry = geometry)

geometry = [Point(xy) for xy in zip(aid.longitude, aid.latitude)]
gaid = gpd.GeoDataFrame(aid, crs = {'init': 'epsg:4326'}, geometry = geometry)

geometry = [Point(xy) for xy in zip(worker_deaths.Longitude, worker_deaths.Latitude)]
g_w_d = gpd.GeoDataFrame(worker_deaths, crs = {'init': 'epsg:4326'}, geometry = geometry) # g_w_d = gworker_deaths


gconflict = gpd.sjoin(gconflict, provinces, how="inner")
gaid = gpd.sjoin(gaid, provinces, how="inner")
g_w_d = gpd.sjoin(g_w_d, provinces, how="inner")


In [425]:
# calculate the number of rows there column_name has more than one entry, separated by "|". We use this for donors and aid focus. 
def calc_multiples(df, column_name):
    s = 0
    l = []
    for index, row in df.iterrows():
        entries = row[column_name].split("|")
        if len(entries) > 1:
            s += 1
            l.append(index)
    return s#, l

# split those rows that have more than one entry in a certain column name.
# We create a new row for each of the multiple entries (other entries being the same)
# and delete the original multiple entry row. 
def split_rows(data, column_name):
    df = copy.deepcopy(data)
    temp = pd.DataFrame()
    for index, row in df.iterrows():
        entries = row[column_name].split("|")
        if len(entries) > 1:
            for entry in entries:
                temp_row = row
                temp_row[column_name] = entry
                temp = temp.append(temp_row)
            df.drop(index, inplace = True)
            
    return df.append(temp)

# split the rows(projects) that have more than one donor per project
gaid = split_rows(gaid, 'donors')
gaid = split_rows(gaid, 'ad_sector_names')

In [426]:
if not os.path.exists('measures_indices'):
    os.makedirs('measures_indices')
    
# how many projects there are per donor and their share in the total, save in a csv file  
total = gaid['donors'].value_counts() # total N of projects per donor
share = gaid['donors'].value_counts() / gaid.shape[0] # calculate share of the projects per donor in total N of projects
pd.concat([total, share], axis = 1).to_csv('measures_indices\\N_projects_per_donor.csv')

# what share of projects per donor have multiple focus sectors (e.g. General environmental protection|Transport and storage)
multiple_focus_share = (gaid.groupby('donors').apply(calc_multiples, 'ad_sector_names') / gaid['donors'].value_counts())
multiple_focus_share.to_csv('measures_indices\\project_multiple_sectors.csv')

# share of each project focus in the total number of projects per donor. We use this to calculate variablity of project focus for a donor
focus_share = (gaid.groupby(['donors', 'ad_sector_names']).size() / gaid.groupby(['donors']).size())
focus_share.to_csv('measures_indices\\donor_sector_share.csv')

In [427]:
# Func calculates the coefficient of unalikeability (as defined by Kader 2007) of every sublcass of first_group variable by topic_name
# E.g. unalikeability of project location for each aid donor -> calc_unalikeability(gaid, 'donors', 'ad_sector_names')

def calc_unalikeability(data, first_group, topic_name):
    
    # prepare data: group by first_group and topic_name, and divide by the size of the respective group
    # thus we obtain the share that each topic_name has in the respective first_group
    
    d = (data.groupby([first_group, topic_name]).size() / gaid.groupby([first_group]).size())
    
    # here we get the keys for the first level grouping. So, the unique values of first group column
    keys = []
    for i in d.index:
        keys.append(i[0])
    keys = set(keys)
    
    # here we calculate the actual coefficient
    # for every value of first_group we calculate its coefficient:
    # coefficient is defined as 1 - SUM_i(p_i^2), where p_i is the share of the ith subgroup in the total group. 
    
    coefs = {}
    for key in keys:
        s = 0
        for subgroup in d[key]:
            s += subgroup ** 2
        coef = 1 - s
        coefs[key] = coef
    return pd.Series(coefs)


# calculates the variablity of topic_name (e.g. total commitments of money) for every member of first_group
def calc_var(data, first_group, topic_name):
    d = data.groupby(first_group)
    
    def var(d):
        d = d[topic_name]
        d = (d - min(d)) / (max(d) - min(d))
        
        if np.isnan(np.var(d)):
            return 0
        
        return np.var(d)
    
    return d.apply(var)

In [428]:
adaptability_by_focus = calc_unalikeability(gaid, 'donors', 'ad_sector_names')
adaptability_by_location =calc_unalikeability(gaid, 'donors', 'NAME_1')
adaptability_by_start_year = calc_unalikeability(gaid, 'donors', 'transactions_start_year')
adaptability_by_commitment = calc_var(gaid, 'donors', 'total_commitments')
composite_adaptability = adaptability_by_commitment + adaptability_by_start_year + adaptability_by_location + adaptability_by_focus
#pd.DataFrame.from_dict(adaptability_by_focus, orient = 'index').to_csv('measures_indices\\adaptability_by_focus.csv')
#pd.DataFrame.from_dict(adaptability_by_location, orient = 'index').to_csv('measures_indices\\adaptability_by_location.csv')
#pd.DataFrame.from_dict(adaptability_by_start_year, orient = 'index').to_csv('measures_indices\\adaptability_by_start_year.csv')
#adaptability_by_commitment.to_csv('measures_indices\\adaptability_by_commitments.csv')
#(composite_adaptability).to_csv('measures_indices\\composite_adaptability.csv')

In [429]:
# Divide donors into 2 groups: higly adaptable and not higly adaptable, the division line is the median. 
median = composite_adaptability.median()
high_adaptability = composite_adaptability[composite_adaptability >= median]
low_adaptability = composite_adaptability[composite_adaptability < median]

In [430]:
# Compute the year, ADM1 pairs of events when aid workers were killed. 
# Then, we can assume that these are unreceptive year, region pairs.
# However, we can also divide this into 2 subgroups: more than average (or median) casualties vs less than average (or median) casualties
worker_deaths_grouped = g_w_d.groupby(['Year', 'NAME_1'])['Total affected'].sum()
worker_deaths_grouped.to_csv('measures_indices\\year_location_pairs.csv')

# The idea is that low_receptivity is depcited by a lot of worker deaths
# high receptivity is depicted by a lesser number of worker deaths
low_receptivity = g_w_d.groupby(['Year', 'NAME_1'])['Total affected'].sum() > worker_deaths_grouped.describe()['mean']
high_receptivity = g_w_d.groupby(['Year', 'NAME_1'])['Total affected'].sum() < worker_deaths_grouped.describe()['mean']


In [431]:
# custom function to check whether a tuple is included in an array of tuples (for some reason, basic function did not work)
def check_in (tupl, array):
    for element in array:
        if (tupl[0] == element[0]) & (tupl[1] == element[1]):
            return True

In [432]:
# select those row numbers (indices) of gaid dataframe that represent projects that started in the same year/adm1 combination as highly receptive ones
high_receptive_indices = []
for index, row in gaid.iterrows():
    tupl = (int(row['transactions_start_year']), row['NAME_1'])
    if check_in(tupl, worker_deaths_grouped[high_receptivity].index.values):
        high_receptive_indices.append(index)

        
# select projects that are created in higly receptive and low receptive year/adm1 combinations 
highly_receptive = gaid.iloc[high_receptive_indices]
low_receptive = gaid.loc[~gaid.index.isin(high_receptive_indices)]

In [433]:
# High receptivity + high adaptability indices
high_receptivity_high_adaptability = highly_receptive[highly_receptive['donors'].isin(high_adaptability.index)]
high_receptivity_low_adaptability = highly_receptive[highly_receptive['donors'].isin(low_adaptability.index)]

low_receptivity_high_adaptability = low_receptive[low_receptive['donors'].isin(high_adaptability.index)]
low_receptivity_low_adaptability = low_receptive[low_receptive['donors'].isin(low_adaptability.index)]

In [14]:
print(high_receptivity_high_adaptability.shape)
print(high_receptivity_low_adaptability.shape)
print(low_receptivity_high_adaptability.shape)
print(low_receptivity_low_adaptability.shape)

(105, 44)
(22, 44)
(1904, 44)
(260, 44)


In [435]:
four_groups = [high_receptivity_high_adaptability, high_receptivity_low_adaptability]

SyntaxError: invalid token (<ipython-input-435-475bb3552a12>, line 1)

In [412]:
region_year_casualties = gconflict[['year', 'NAME_1', 'best_est']].groupby(['NAME_1', 'year']).sum()
region_year_aid = gaid[['transactions_start_year', 'NAME_1', 'even_split_commitments']].groupby(['NAME_1', 'transactions_start_year']).sum()

In [413]:
def add_lag(df, column_name, new_name):
    output = pd.DataFrame()
    df[new_name] = df.groupby(level = 0)[column_name].shift(1)
    by_country = df.groupby(level = 0)
    for name, group in by_country:
        country = group.reset_index()
        last_row = country.iloc[-1, :]
        if new_name == 'lagged_commitments':
            temp = pd.Series({'NAME_1': last_row['NAME_1'],
                      'transactions_start_year': last_row['transactions_start_year'] + 1, 
                      'even_split_commitments': float('nan'),
                      'lagged_commitments' : last_row['even_split_commitments']})
        if new_name == 'lagged_casualties':
            temp = pd.Series({'NAME_1': last_row['NAME_1'],
                      'year': last_row['year'] + 1, 
                      'best_est': float('nan'),
                      'lagged_casualties' : last_row['best_est']})
        country = country.append(temp, ignore_index = True)
        output = output.append(country)
    return output

In [414]:
# add the lag variable (for a (district, year) pair (d, t) , we are looking at commitments on (d, t-1). This action also resets hierachical index
region_year_aid = add_lag(region_year_aid, 'even_split_commitments', 'lagged_commitments')

# rename the column regarding the start year so that it corresponds to the name in casualties dataset
region_year_aid = region_year_aid.rename(columns = {'even_split_commitments': 'commitments', 'transactions_start_year': 'year'})

# reset the hierachical index, so that the dataframe obtain the same structure
region_year_casualties = add_lag(region_year_casualties, 'best_est', 'lagged_casualties')

In [415]:
region_year_aid['commitments_difference'] = region_year_aid['commitments'] - region_year_aid['lagged_commitments']
region_year_aid['log_commitments'] = np.log(region_year_aid['commitments'] )
region_year_aid['log_lagged_commitments'] = np.log(region_year_aid['lagged_commitments'] )
region_year_aid['log_difference'] = np.log(region_year_aid['commitments_difference'] )
region_year_aid['difference_of_logs'] = region_year_aid['log_commitments'] - region_year_aid['log_lagged_commitments']

region_year_casualties['casulaties_difference'] = region_year_casualties['best_est'] - region_year_casualties['lagged_casualties']
region_year_casualties['casulaties_log'] = np.log(region_year_casualties['best_est'])
#region_year_casualties['casulaties_log'] = np.log(region_year_casualties['best_est'])
region_year_casualties['casulaties_standardized'] = (region_year_casualties['best_est'] - np.mean(region_year_casualties['best_est'])) / np.var(region_year_casualties['best_est'])

C:\Users\Yurii\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in log
  after removing the cwd from sys.path.
C:\Users\Yurii\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log
  


In [416]:
aid_and_conflict = pd.merge(region_year_aid, region_year_casualties, on = ['NAME_1', 'year'])
aid_and_conflict

,NAME_1,year,commitments,lagged_commitments,commitments_difference,log_commitments,log_lagged_commitments,log_difference,difference_of_logs,best_est,lagged_casualties,casulaties_difference,casulaties_log,casulaties_standardized
0,Bas-Uélé,1999,3.812963e+06,NaN,NaN,15.153917,NaN,NaN,NaN,100.0,217.0,-117.0,4.605170,-0.000118
1,Bas-Uélé,2006,3.506466e+07,3.812963e+06,3.125170e+07,17.372704,15.153917,17.257584,2.218787,1.0,22.0,-21.0,0.000000,-0.000155
2,Bas-Uélé,2008,4.492892e+07,5.684461e+06,3.924445e+07,17.620592,15.553247,17.485321,2.067345,69.0,1.0,68.0,4.234107,-0.000130
3,Bas-Uélé,2009,3.422347e+07,4.492892e+07,-1.070544e+07,17.348422,17.620592,NaN,-0.272170,230.0,69.0,161.0,5.438079,-0.000071
4,Bas-Uélé,2010,2.048740e+07,3.422347e+07,-1.373607e+07,16.835321,17.348422,NaN,-0.513102,116.0,230.0,-114.0,4.753590,-0.000113
5,Bas-Uélé,2011,1.878535e+07,2.048740e+07,-1.702047e+06,16.748588,16.835321,NaN,-0.086733,49.0,116.0,-67.0,3.891820,-0.000137
6,Bas-Uélé,2012,3.360065e+07,1.878535e+07,1.481529e+07,17.330056,16.748588,16.511171,0.581468,1.0,49.0,-48.0,0.000000,-0.000155
7,Bas-Uélé,2013,9.944423e+06,3.360065e+07,-2.365623e+07,16.112522,17.330056,NaN,-1.217533,12.0,1.0,11.0,2.484907,-0.000151
8,Bas-Uélé,2014,NaN,9.944423e+06,NaN,NaN,16.112522,NaN,NaN,26.0,12.0,14.0,3.258097,-0.000145
9,Haut-Katanga,2008,1.854662e+07,4.207627e+07,-2.352965e+07,16.735798,17.554994,NaN,-0.819196,1.0,3.0,-2.0,0.000000,-0.000155


In [419]:
result = sm.ols(formula = 'best_est ~ log_lagged_commitments', data = aid_and_conflict).fit()

In [420]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               best_est   R-squared:                       0.048
Model:                            OLS   Adj. R-squared:                  0.034
Method:                 Least Squares   F-statistic:                     3.484
Date:                Sat, 07 Jul 2018   Prob (F-statistic):             0.0662
Time:                        23:05:27   Log-Likelihood:                -532.14
No. Observations:                  71   AIC:                             1068.
Df Residuals:                      69   BIC:                             1073.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept               -984.8242    629.546     -1.564      0.122   -2240.735     271.087
log_lagged_commitments    71.7034     38.414      1.867      0.066      -4.930     148.337
==============================================================================
Omnibus:                       84.526   Durbin-Watson:                   1.244
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              859.533
Skew:                           3.666   Prob(JB):                    2.26e-187
Kurtosis:                      18.388   Cond. No.                         198.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
# TODO LIST

# 3. Select projects based on the 4 subcategories derived above.
# Projects with: donor is in one of the first classification group.
# Start year and area is in one of the second classification groups.

#                        High adaptability              Low adaptability
#
# High receptivity           n                                 n2
#
# Low receptivity            n3                                n4



# 4. THINK ABOUT INFERENCE PROBLEM, MATCHING PROBLEM, CREATING CONTROL AND TREATMENT GROUPS.